In [1]:
import opendatasets as od


od.download("https://www.kaggle.com/datasets/mostafaabla/garbage-classification")

Skipping, found downloaded files in ".\garbage-classification" (use force=True to force download)


In [2]:
import torch
import sys
import platform
has_gpu = torch.cuda.is_available()
# has_mps = getattr(torch,'has_mps',False)
has_mps = torch.backends.mps.is_built()
device = "mps" if torch.backends.mps.is_built() \
    else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Python {sys.version}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AzVAILABLE")
print(f"Target device is {device}")

Python Platform: Windows-10-10.0.22621-SP0
PyTorch Version: 2.1.1+cu121
Python 3.11.0 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:21) [MSC v.1916 64 bit (AMD64)]
NVIDIA/CUDA GPU is available
MPS (Apple Metal) is NOT AzVAILABLE
Target device is cuda


In [3]:
import pathlib
import numpy as np
dataPath = pathlib.Path.cwd() / "garbage-classification"

print(dataPath)

isFile = lambda path : path.is_file()

imagePaths = list(filter(isFile, list(dataPath.rglob("*"))))

np.random.shuffle(imagePaths)

#pathSamples = np.random.choice(imagePaths, 8000)



c:\Users\sfaub\OneDrive\Desktop\CodeJam\CodeJam13\model\garbage-classification


In [4]:
from PIL import Image
import torchvision.transforms.v2 as v2

torch.cuda.empty_cache()

transformer = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.Resize(size=(224, 224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

trainInputImages = []

validateInputImages = []

testInputImages = []

trainLabelsList = []

validateLabelsList = []

testLabelsList = []

rejected = []

for index in range(len(imagePaths)):
    pathStr = str(imagePaths[index])
    labelNum = -1
    if 'battery' in pathStr:
        labelNum = 0
    elif 'biological' in pathStr:
        labelNum = 1
    elif 'brown-glass' in pathStr:
        labelNum = 2
    elif 'cardboard' in pathStr:
        labelNum = 3
    elif 'clothes' in pathStr:
        continue
    elif 'green-glass' in pathStr:
        labelNum = 4
    elif 'metal' in pathStr:
        labelNum = 5
    elif 'paper' in pathStr:
        labelNum = 6
    elif 'plastic' in pathStr:
        labelNum = 7
    elif 'shoes' in pathStr:
        continue
    elif 'trash' in pathStr:
        labelNum = 8
    elif 'white-glass' in pathStr:
        labelNum = 9
    if labelNum == -1:
        continue
    img = Image.open(pathStr).convert('RGB')
    tensor = transformer(img)
    if tensor.shape[0] != 3:
        rejected.append(pathStr)
        continue
    if index < int(0.8*len(imagePaths)):
        trainLabelsList.append(labelNum)
        trainInputImages.append(tensor)
    elif index < int(0.8*len(imagePaths) + 0.1*len(imagePaths)):
        validateLabelsList.append(labelNum)
        validateInputImages.append(tensor)
    else:
        testLabelsList.append(labelNum)
        testInputImages.append(tensor)    
    


trainDataset = torch.stack(trainInputImages)
trainLabels = torch.ByteTensor(trainLabelsList)

print(trainDataset.shape)
print(trainLabels.shape)


torch.Size([6564, 3, 224, 224])
torch.Size([6564])


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.dropout = nn.Dropout(0.7)
        self.conv1 = nn.Conv2d(3, 20, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(20, 50, kernel_size=(5, 5))
        self.fc1 = nn.Linear(50*53*53, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.logSoftMax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.logSoftMax(x)

In [15]:
batch_size = 64

model = CNN().to(device)

criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(20):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[i:i+batch_size].to(device), trainLabels[i:i+batch_size].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, curLabels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainDataset)}")
    
print("Finished Training")


Epoch 1, Loss: 0.05191609130844265
Epoch 2, Loss: 0.03215497430688648
Epoch 3, Loss: 0.029171770062844684
Epoch 4, Loss: 0.02726331649213067
Epoch 5, Loss: 0.025446601407256235
Epoch 6, Loss: 0.02396255709789353
Epoch 7, Loss: 0.022329329282027202
Epoch 8, Loss: 0.021291703385307177
Epoch 9, Loss: 0.020555789408198515
Epoch 10, Loss: 0.01919362228647519
Epoch 11, Loss: 0.01872756094703
Epoch 12, Loss: 0.018267412426484785
Epoch 13, Loss: 0.017275603822932455
Epoch 14, Loss: 0.016580390565138194
Epoch 15, Loss: 0.016049251541504404
Epoch 16, Loss: 0.015384963904845141
Epoch 17, Loss: 0.015381955350411512
Epoch 18, Loss: 0.015017369429488359
Epoch 19, Loss: 0.014313582156141817
Epoch 20, Loss: 0.01390664457439432
Finished Training


In [16]:

# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[index:index+batch_size].to(device), trainLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the train set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the train set: 81.70%


In [17]:

validateDataset = torch.stack(validateInputImages)
validateLabels = torch.ByteTensor(validateLabelsList)
# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(validateDataset), batch_size):
        inputs, curLabels = validateDataset[index:index+batch_size].to(device), validateLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the validation set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the validation set: 60.09%


In [18]:

testDataset = torch.stack(testInputImages)
testLabels = torch.ByteTensor(testLabelsList)
# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(testDataset), batch_size):
        inputs, curLabels = testDataset[index:index+batch_size].to(device), testLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the test set: 58.23%


In [20]:
torch.save(model.state_dict(), pathlib.Path.cwd() / 'model_1.pt')

In [21]:
model.train()

for epoch in range(20):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[i:i+batch_size].to(device), trainLabels[i:i+batch_size].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, curLabels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+9}, Loss: {running_loss / len(trainDataset)}")
    
print("Finished Training")


Epoch 9, Loss: 0.01378597620599957
Epoch 10, Loss: 0.013325799281534663
Epoch 11, Loss: 0.012286025025109414
Epoch 12, Loss: 0.012413067253968253
Epoch 13, Loss: 0.012340471788668763
Epoch 14, Loss: 0.011664982419744778
Epoch 15, Loss: 0.011559715999618381
Epoch 16, Loss: 0.011676222721590348
Epoch 17, Loss: 0.010882926851710774
Epoch 18, Loss: 0.010927187409399486
Epoch 19, Loss: 0.011246479421145737
Epoch 20, Loss: 0.01052427662810053
Epoch 21, Loss: 0.010446096487266127
Epoch 22, Loss: 0.010098035332090168
Epoch 23, Loss: 0.009763605747369642
Epoch 24, Loss: 0.009874290210636995
Epoch 25, Loss: 0.009520403645810088
Epoch 26, Loss: 0.009647335950228443
Epoch 27, Loss: 0.009136708098021605
Epoch 28, Loss: 0.008986311004991433
Finished Training


In [22]:

# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[index:index+batch_size].to(device), trainLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the train set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the train set: 92.43%


In [23]:

validateDataset = torch.stack(validateInputImages)
validateLabels = torch.ByteTensor(validateLabelsList)
# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(validateDataset), batch_size):
        inputs, curLabels = validateDataset[index:index+batch_size].to(device), validateLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the validation set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the validation set: 59.62%


In [24]:

testDataset = torch.stack(testInputImages)
testLabels = torch.ByteTensor(testLabelsList)
# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(testDataset), batch_size):
        inputs, curLabels = testDataset[index:index+batch_size].to(device), testLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the test set: 58.48%


In [25]:
torch.save(model.state_dict(), pathlib.Path.cwd() / 'model_2.pt')

In [26]:
model.train()

for epoch in range(20):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[i:i+batch_size].to(device), trainLabels[i:i+batch_size].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, curLabels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch}, Loss: {running_loss / len(trainDataset)}")
    
print("Finished Training")


Epoch 0, Loss: 0.008532180437803414
Epoch 1, Loss: 0.008912262953431921
Epoch 2, Loss: 0.008606898446841475
Epoch 3, Loss: 0.008783273253587598
Epoch 4, Loss: 0.008594577225114716
Epoch 5, Loss: 0.008402563915301357
Epoch 6, Loss: 0.008104547581281842
Epoch 7, Loss: 0.008012445696892351
Epoch 8, Loss: 0.00815344656352805
Epoch 9, Loss: 0.008102227921786649
Epoch 10, Loss: 0.00782561153264252
Epoch 11, Loss: 0.0075299883037714755
Epoch 12, Loss: 0.007939996544015183
Epoch 13, Loss: 0.007501538636334272
Epoch 14, Loss: 0.007803875864423825
Epoch 15, Loss: 0.007396730894562241
Epoch 16, Loss: 0.007533089865605182
Epoch 17, Loss: 0.007256252957890485
Epoch 18, Loss: 0.007451622095923828
Epoch 19, Loss: 0.007202051927282779
Finished Training


In [28]:
np.random.shuffle(imagePaths)

trainInputImages = []

validateInputImages = []

testInputImages = []

trainLabelsList = []

validateLabelsList = []

testLabelsList = []

rejected = []

for index in range(len(imagePaths)):
    pathStr = str(imagePaths[index])
    labelNum = -1
    if 'battery' in pathStr:
        labelNum = 0
    elif 'biological' in pathStr:
        labelNum = 1
    elif 'brown-glass' in pathStr:
        labelNum = 2
    elif 'cardboard' in pathStr:
        labelNum = 3
    elif 'clothes' in pathStr:
        continue
    elif 'green-glass' in pathStr:
        labelNum = 4
    elif 'metal' in pathStr:
        labelNum = 5
    elif 'paper' in pathStr:
        labelNum = 6
    elif 'plastic' in pathStr:
        labelNum = 7
    elif 'shoes' in pathStr:
        continue
    elif 'trash' in pathStr:
        labelNum = 8
    elif 'white-glass' in pathStr:
        labelNum = 9
    if labelNum == -1:
        continue
    img = Image.open(pathStr).convert('RGB')
    tensor = transformer(img)
    if tensor.shape[0] != 3:
        rejected.append(pathStr)
        continue
    if index < int(0.8*len(imagePaths)):
        trainLabelsList.append(labelNum)
        trainInputImages.append(tensor)
    elif index < int(0.8*len(imagePaths) + 0.1*len(imagePaths)):
        validateLabelsList.append(labelNum)
        validateInputImages.append(tensor)
    else:
        testLabelsList.append(labelNum)
        testInputImages.append(tensor)    
    


trainDataset = torch.stack(trainInputImages)
trainLabels = torch.ByteTensor(trainLabelsList)

In [29]:
model.train()

for epoch in range(20):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[i:i+batch_size].to(device), trainLabels[i:i+batch_size].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, curLabels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch}, Loss: {running_loss / len(trainDataset)}")
    
print("Finished Training")


Epoch 0, Loss: 0.012511615788406311
Epoch 1, Loss: 0.011149287291919946
Epoch 2, Loss: 0.009994607316619518
Epoch 3, Loss: 0.009894531785461106
Epoch 4, Loss: 0.008863523162079614
Epoch 5, Loss: 0.008828155196962525
Epoch 6, Loss: 0.008237722362144774
Epoch 7, Loss: 0.007817404111989164
Epoch 8, Loss: 0.007812083718672822
Epoch 9, Loss: 0.007689199029563913
Epoch 10, Loss: 0.007697748350595861
Epoch 11, Loss: 0.00702165092167369
Epoch 12, Loss: 0.00726652832782392
Epoch 13, Loss: 0.006826700773237683
Epoch 14, Loss: 0.006419704258805147
Epoch 15, Loss: 0.006772400106437899
Epoch 16, Loss: 0.006751126973531654
Epoch 17, Loss: 0.006708882833075625
Epoch 18, Loss: 0.0066142819803673494
Epoch 19, Loss: 0.006691806311199826
Finished Training


In [30]:

# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(trainDataset), batch_size):
        inputs, curLabels = trainDataset[index:index+batch_size].to(device), trainLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the train set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the train set: 96.54%


In [31]:

validateDataset = torch.stack(validateInputImages)
validateLabels = torch.ByteTensor(validateLabelsList)
# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(validateDataset), batch_size):
        inputs, curLabels = validateDataset[index:index+batch_size].to(device), validateLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the validation set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the validation set: 82.96%


In [32]:

testDataset = torch.stack(testInputImages)
testLabels = torch.ByteTensor(testLabelsList)
# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(0, len(testDataset), batch_size):
        inputs, curLabels = testDataset[index:index+batch_size].to(device), testLabels[index:index+batch_size].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the test set: 87.72%


In [ ]:
torch.save(model.state_dict(), pathlib.Path.cwd() / 'model_3.pt')